<a href="https://colab.research.google.com/github/Yugant1338/Indian_Law_Bot/blob/main/LawGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets gradio kaggle sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.3 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d anishparkhe0401/indian-laws
!unzip -q indian-laws.zip -d indian_laws

!kaggle datasets download -d omdabral/indian-penal-code-complete-dataset
!unzip -q indian-penal-code-complete-dataset.zip -d ipc_data


Dataset URL: https://www.kaggle.com/datasets/anishparkhe0401/indian-laws
License(s): MIT
  0% 0.00/358k [00:00<?, ?B/s]
100% 358k/358k [00:00<00:00, 814MB/s]
Dataset URL: https://www.kaggle.com/datasets/omdabral/indian-penal-code-complete-dataset
License(s): unknown
  0% 0.00/79.2k [00:00<?, ?B/s]
100% 79.2k/79.2k [00:00<00:00, 163MB/s]


In [4]:
import os
import json
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import gradio as gr


In [5]:
# Load law sections
with open("/content/indian_laws/combined.json", "r") as f:
    law_data = json.load(f)

# Load IPC dataset
ipc_df = pd.read_csv("/content/ipc_data/FIR_DATASET.csv").dropna()


In [6]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Combine all context texts (law sections + IPC descriptions)
law_texts = [entry["description"] for entry in law_data]
ipc_texts = ipc_df['Description'].tolist()
all_contexts = law_texts + ipc_texts

# Embed all contexts
context_embeddings = embedder.encode(all_contexts, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
def find_semantic_context(question):
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, context_embeddings, top_k=1)
    best_index = hits[0][0]['corpus_id']
    return all_contexts[best_index]


In [9]:
def ask_law_bot(question):
    context = find_semantic_context(question)
    prompt = f"Question: {question}\nContext: {context}\nAnswer:"
    result = qa_pipeline(prompt, max_length=256)[0]["generated_text"]
    return result


In [10]:
gr.Interface(
    fn=ask_law_bot,
    inputs="text",
    outputs="text",
    title="⚖️ Indian Legal Chatbot (RAG + Generative)",
    description="Ask questions about IPC or Indian law acts. Powered by semantic search and FLAN-T5."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f97969e296d86d1ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
